# Parole Models: Initial data exploration, cleaning, & processing

### Willie Costello

In [1]:
import numpy as np
import pandas as pd

## Importing & cleaning data

In [2]:
df_raw = pd.read_csv('~/data/ParoleModels/data.csv')

In [3]:
df_raw.shape

(744958, 26)

In [4]:
# Check for duplicates
df_raw.duplicated().sum()

0

In [5]:
# Check for missing values
df_raw.isna().mean().sort_values()

FISCAL YEAR                     0.000000
WARRANT ID                      0.000000
LOCATION TYPE                   0.000000
PROVINCE                        0.000000
OFFENCE ID                      0.000000
SENTENCE TYPE                   0.000000
JURISDICTION                    0.000000
OFFENCE DESCRIPTION             0.000000
AGE                             0.000000
GENDER                          0.000000
RACE GROUPING                   0.000000
OFFENDER NUMBER                 0.000000
IN CUSTODY/COMMUNITY            0.000000
SENTENCE ID                     0.000000
AGGREGATE SENTENCE LENGTH       0.000008
COURT                           0.014460
RACE                            0.017673
STATIC/RISK                     0.029078
DYNAMIC/NEED                    0.029407
REINTEGRATION POTENTIAL         0.030132
MOTIVATION                      0.030594
OFFENDER SECURITY LEVEL         0.065860
RELIGION                        0.070193
INSTUTUTIONAL SECURITY LEVEL    0.317110
SUPERVISION TYPE

`JUDGE` column is 100% missing values, so we will drop it. We will also drop `INSTUTUTIONAL SECURITY LEVEL` and `SUPERVISION TYPE`, due to their high percentage of missing values.

In [6]:
df_clean = df_raw.drop(['JUDGE', 'SUPERVISION TYPE', 'INSTUTUTIONAL SECURITY LEVEL'], axis=1)

Following the *Globe*'s analysis, we will only consider data from offenders serving a federal sentence, as indicated in the `JURISDICTION` column.

In [7]:
df_clean = df_clean[df_clean['JURISDICTION']=='FEDERAL']
df_clean.drop('JURISDICTION', axis=1, inplace=True)

Let's next convert the `FISCAL YEAR` column into something more descriptive.

In [8]:
df_clean['YEAR'] = df_clean['FISCAL YEAR'].apply(lambda x: '20' + x[-2:])

Let's now convert some of the two-category columns into binary columns.

In [9]:
df_clean['MAN'] = np.where(df_clean['GENDER']=='MALE', 1, 0)
df_clean['INDIGENOUS'] = np.where(df_clean['RACE GROUPING']=='Indigenous', 1, 0)
df_clean['IN_CUSTODY'] = np.where(df_clean['IN CUSTODY/COMMUNITY']=='In Custody', 1, 0)
df_clean['DETERMINATE_SENTENCE'] = np.where(df_clean['SENTENCE TYPE']=='DETERMINATE', 1, 0)

In [10]:
# Round Offender Number to five decimal points, to fix some datatype issues
df_clean['OFFENDER ID'] = df_clean['OFFENDER NUMBER'].apply(lambda x: round(x, 5))

In [11]:
# Drop converted columns
df_clean.drop(['FISCAL YEAR', 'GENDER', 'RACE GROUPING', 'IN CUSTODY/COMMUNITY', 'SENTENCE TYPE', 'OFFENDER NUMBER'], axis=1, inplace=True)

In [12]:
# Reformat column names
df_clean.columns = [x.lower().replace(' ', '_') for x in df_clean.columns]

In [13]:
df_clean.shape

(741829, 22)

In [14]:
df_clean.head()

,sentence_id,race,age,aggregate_sentence_length,province,location_type,offender_security_level,dynamic/need,static/risk,reintegration_potential,...,warrant_id,court,offence_id,offence_description,year,man,indigenous,in_custody,determinate_sentence,offender_id
0,U40A00014615,White,23,1096.0,ONTARIO,FEDERAL INSTITU,MINIMUM,MEDIUM,LOW,HIGH,...,U40A00081680,ONT COURT OF JUSTICE,U40A00073751,POSS PROH/RESTR F/ARM W/AMMUN,2012,1,0,1,1,82.50710
1,U40A00014615,White,23,1096.0,ONTARIO,FEDERAL INSTITU,MINIMUM,MEDIUM,LOW,HIGH,...,U40A00081680,ONT COURT OF JUSTICE,U40A00073752,FAIL TO COMPLY W/ PROBATION ORDER,2012,1,0,1,1,82.50710
2,U40A00014615,White,23,1096.0,ONTARIO,FEDERAL INSTITU,MINIMUM,MEDIUM,LOW,HIGH,...,U40A00081680,ONT COURT OF JUSTICE,U40A00073753,POSS SCHEDULE I/II SUBST FOR PURP TRAFF,2012,1,0,1,1,82.50710
3,U80A00008369,White,39,732.0,BRITISH COLUMBI,FEDERAL INSTITU,MEDIUM,HIGH,HIGH,LOW,...,U80A00048789,VANCOUVER PROVINCIAL COURT,U80A00029774,ASSAULT - USE OF FORCE,2012,1,0,1,1,104.76347
4,U80A00008369,White,39,732.0,BRITISH COLUMBI,FEDERAL INSTITU,MEDIUM,HIGH,HIGH,LOW,...,U80A00048789,VANCOUVER PROVINCIAL COURT,U80A00029785,ROBBERY - ALL OTHERS,2012,1,0,1,1,104.76347


In [15]:
# Create a copy of cleaned dataframe
df = df_clean.copy()

# Filter to only men
df = df[df['man']==1].reset_index(drop=True)

# Filter to only relevant columns
df = df_clean[['offender_id', 'year', 'indigenous', 'age', 'offence_description', 'static/risk', 'determinate_sentence', 'reintegration_potential']].copy()

## Exploring column values

In [16]:
for col in df.select_dtypes(object).columns:
    print(f'Value counts for {col}\n')
    print(df[col].value_counts())
    print('\n-------------------------\n')

Value counts for year

2012    107918
2013    107243
2014    107093
2015    106059
2016    106014
2017    104638
2018    102864
Name: year, dtype: int64

-------------------------

Value counts for offence_description

ROBBERY - ALL OTHERS                        36764
FAIL TO COMPLY W/COND OF UNDER/RECOG        33660
POSS SCHEDULE I/II SUBST FOR PURP TRAFF     29807
BREAK ENTER AND COMMIT                      29245
FAIL TO COMPLY W/ PROBATION ORDER           26450
                                            ...  
SEX ASSAULT W/THREATS TO CBH- FIREARM           1
HIJACK - CONFINE PERSON                         1
BREACH OF CONTRACT TO CAUSE INJURY              1
SEXUAL ASSAULT-PARTY TO OFFENCE- FIREARM        1
THEFT OF TELECOMMUNICATION SERV - UNDER         1
Name: offence_description, Length: 739, dtype: int64

-------------------------

Value counts for static/risk

HIGH      378885
MEDIUM    251230
LOW        90098
Name: static/risk, dtype: int64

-------------------------

Value cou

## Transforming static/risk column

In [17]:
# Lowercase column values
df['static/risk'] = df['static/risk'].str.lower()

In [18]:
# Dummify static/risk column, dropping medium risk column
static_dummies = pd.get_dummies(df['static/risk'], prefix='static').drop('static_medium', axis=1)

In [19]:
# Add dummy columns to dataframe & remove original column
df = pd.concat([df, static_dummies], axis=1).drop('static/risk', axis=1)

## Filtering to top offences

In [20]:
# Lowercase column values & replace whitespaces with underscores
df['offence_description'] = df['offence_description'].str.lower().str.replace(' ', '_')

In [21]:
# Get normalized value counts for all offence descriptions
offence_pcts = df['offence_description'].value_counts(normalize=True)

In [22]:
# Filter to offence descriptions that appear in more than 0.1% of the data
offence_pcts[offence_pcts > .001]

robbery_-_all_others                       0.049559
fail_to_comply_w/cond_of_under/recog       0.045374
poss_schedule_i/ii_subst_for_purp_traff    0.040180
break_enter_and_commit                     0.039423
fail_to_comply_w/_probation_order          0.035655
                                             ...   
conspire_to_commit_murder                  0.001073
discharge_rest/proh_firearm_w/intent       0.001037
prison_breach_with_intent                  0.001035
impaired_driving_-_cbh                     0.001014
production_of_sched_i/ii_subst             0.001011
Name: offence_description, Length: 136, dtype: float64

In [23]:
# Create a list of these top offences
top_offences = offence_pcts[offence_pcts > .001].index.tolist()

In [24]:
# Filter dataframe to only rows with one of these top offences
df_filtered = df[df['offence_description'].isin(top_offences)].reset_index(drop=True)

## Transforming data

In [25]:
# Dummify offence description column
offence_dummies = pd.get_dummies(df_filtered['offence_description'], prefix='offence')

There are two ways we may want to aggregate our dummified offence description columns: We can either record the inmate's total number of charges for each offence type, or we can simply record whether or not the inmate was charged with each offence type (regardless of the total number of charges). It's not clear to me which method will be better for modelling purposes, so I'll here do both.

In [26]:
# Select all columns other than Offence Description and Reintegration Potential (target variable)
df_others = df_filtered.drop(['offence_description', 'reintegration_potential'], axis=1)

In [27]:
# Group dataframe by Offender ID and year (i.e., one row for each unique reintegration potential assessment)
# All values are the same across each set of grouped rows, so taking the max will retrieve those values
df_others_grouped = df_others.groupby(['offender_id', 'year']).max()

In [29]:
# Create dataframe of dependent variable, similarly grouped
df_y_grouped = df_filtered.groupby(['offender_id', 'year'])['reintegration_potential'].max()

# Reset index
y = df_y_grouped.reset_index(drop=True)

### Summing strategy

In [30]:
# Add dummy columns to dataframe's Offender ID & year columns
df_off_dummies = pd.concat([df_filtered[['offender_id', 'year']], offence_dummies], axis=1)

# Group dataframe and take the sum
df_off_sum = df_off_dummies.groupby(['offender_id', 'year']).sum()

# Add summed dummy columns to others
df_X_sum = pd.concat([df_others_grouped, df_off_sum], axis=1)

# Reset index
X_sum = df_X_sum.reset_index(drop=True)

### Binary strategy

In [31]:
# Group dataframe and take the max
df_off_binary = df_off_dummies.groupby(['offender_id', 'year']).max()

# Add binary dummy columns to others
df_X_binary = pd.concat([df_others_grouped, df_off_binary], axis=1)

# Reset index
X_binary = df_X_binary.reset_index(drop=True)

In [32]:
# Write dataframes to file
X_sum.to_csv('~/data/ParoleModels/X_sum.csv', index=False)
X_binary.to_csv('~/data/ParoleModels/X_binary.csv', index=False)
y.to_csv('~/data/ParoleModels/y.csv', index=False)